# Environmental Justice 

This notebook's focus is on racial disparities across counties. 

Contents:
1. Introduction
2. Visualization
3. AQI and Health Outcomes

# Introduction

## Import Libraries

Run the cell below to load the necessary libraries for our data analysis.

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import scipy.stats as stats
import geopandas as gpd
import folium
import ipywidgets as widgets
import branca
from IPython.display import display

## Data                                                                                                                               

Air Quality Index: A tool used to measure and communicate the level of pollution in a given area. According to Environmental Initiative (https://environmental-initiative.org/news-ideas/what-is-the-air-quality-index/), below are the ranges that correspond to the level of concern: 
- 0-50: Good
- 51-100: Moderate
- 101-150: Unhealthy for Sensitive Groups
- 150-200: Unhealthy
- 201-300: Very unhealthy
- 301-500: Hazardous

In general, an AQI at or below 100 is considered safe for most people. Anything above 100 is considered unhealthy for sensitive groups such as elderly, young children, and immunocompromised groups. Our goal is to analyze these factors and whether an intersectionality exists with health factors related to race. But first, check out the data below which contain daily Air Quality Index (AQI) across counties for 2021.   

< insert longer explanation on disparities when it comes to access to clean atmosphere vs race. Might have  to consider economic factors, proximity to toxic sites. >

## Data preprocessing

Before diving deep into analysis, we will first conduct arguably the most important part of the data science lifecycle which is data cleaning, or data preprocessing. The codes are already provided below with accompanying explanation for what they're doing. For now, all you need to do is run the cells.

In [45]:
# Loading data
aqi2021_df = pd.read_csv("daily_aqi_by_county_2021.csv")  

We want to get the average yearly AQI per county but to do that, we must first extract the year-month per record. The code below uses pd.to_datetime() to convert our column 'Date' into a datetime format so we can easily extract Year-Month values before aggregating. In addition, we dropped 'County Code' as it is not the same as FIPS, which is used to identify a specific county in the US. We will later apply merging techniques to match county to their respective FIPS.

In [46]:
aqi2021_df["Date"] = pd.to_datetime(aqi2021_df["Date"], errors="coerce")
aqi2021_df["Year-Month"] = aqi2021_df["Date"].dt.strftime('%Y-%m')
aqi2021_df = aqi2021_df.drop(columns=['County Code'])
aqi2021_df

,State Name,county Name,State Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,Year-Month
0,Alabama,Baldwin,1,2021-01-01,36,Good,PM2.5,01-003-0010,1,2021-01
1,Alabama,Baldwin,1,2021-01-07,32,Good,PM2.5,01-003-0010,1,2021-01
2,Alabama,Baldwin,1,2021-01-13,55,Moderate,PM2.5,01-003-0010,1,2021-01
3,Alabama,Baldwin,1,2021-01-16,28,Good,PM2.5,01-003-0010,1,2021-01
4,Alabama,Baldwin,1,2021-01-19,58,Moderate,PM2.5,01-003-0010,1,2021-01
...,...,...,...,...,...,...,...,...,...,...
326535,Wyoming,Weston,56,2021-12-27,34,Good,Ozone,56-045-0003,1,2021-12
326536,Wyoming,Weston,56,2021-12-28,35,Good,Ozone,56-045-0003,1,2021-12
326537,Wyoming,Weston,56,2021-12-29,34,Good,Ozone,56-045-0003,1,2021-12
326538,Wyoming,Weston,56,2021-12-30,36,Good,Ozone,56-045-0003,1,2021-12


Check out the monthly average Air Quality Index (AQI) below. (Feel free to uncomment to see the table)

In [47]:
monthly_avg_aqi_df = aqi2021_df.groupby(['State Name', 'county Name', 'Year-Month']).agg({
    "AQI": "mean",  
}).reset_index()

monthly_avg_aqi_df = monthly_avg_aqi_df.rename(columns={'AQI': 'Monthly Average AQI'})
monthly_avg_aqi_df = monthly_avg_aqi_df[["county Name", "State Name", "Monthly Average AQI", "Year-Month"]]
#monthly_avg_aqi_df

Let's aggregate it further to find the highest Air Quality Index (AQI) for each county-state pair in the year 2021. Note: In data science, this process of adjusting data granularity is quite common depending on your objective. In this case, we are making our data more coarse (yearly scale on county state level) because we want to simply analyze, on high level, the effect of factors like race or socioeconomic factor on exposure to environmental hazards. (Again, feel free to uncomment to see the table)

In [48]:
yearly_avg_aqi_df = monthly_avg_aqi_df.groupby(['State Name', 'county Name']).agg({
    "Monthly Average AQI": "mean",   
}).reset_index()

yearly_avg_aqi_df = yearly_avg_aqi_df.rename(columns={"Monthly Average AQI": 'Yearly Average AQI'})
yearly_avg_aqi_df = yearly_avg_aqi_df[["county Name", "State Name", "Yearly Average AQI"]]
#yearly_avg_aqi_df

In [49]:
state_abbreviations = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO",
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA",
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD",
    "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"
}

In [50]:
yearly_avg_aqi_df["STATE"] = yearly_avg_aqi_df["State Name"].apply(lambda x: state_abbreviations[x] if x in state_abbreviations else "NA")
yearly_avg_aqi_df["county Name"] = yearly_avg_aqi_df["county Name"] + " County"
#yearly_avg_aqi_df

Great! Now we can merge into our state_and_county_fips dataset so now we have a FIPS for every county-state pair.

In [51]:
fips_df = pd.read_csv("state_and_county_fips_master.csv")
fips_df = fips_df.dropna()
fips_df = fips_df.rename(columns={"name": "county Name", "state": "STATE"})
fips_df.head(3)

,fips,county Name,STATE
2,1001,Autauga County,AL
3,1003,Baldwin County,AL
4,1005,Barbour County,AL


In [52]:
yearly_avg_aqi_df_wfips = fips_df.merge(yearly_avg_aqi_df, on=["county Name", "STATE"], how="inner")
#yearly_avg_aqi_df_wfips

----

# Visualizing Trends in Arizona and California

In this analysis, we will focus on the states of Arizona and California in examining the relationship between socioeconomic factors, race, and air quality levels (AQI). By mapping the Yearly Average AQI at the county level, we aim to identify patterns and disparities in air quality exposure. We will explore whether socioeconomic factors (specifically income and education level) correlate with higher or lower AQI levels. Furthermore, we will investigate potential racial disparities in air pollution exposure, analyzing whether certain racial or ethnic groups are disproportionately affected by poor air quality. 

Through this study, we seek to uncover any systemic trends that may indicate environmental injustice or disparities in air pollution exposure across different communities in Arizona and California.

## GeoPandas as a mapping tool

In data science, visualization is crucial to recognizing patterns in data. While bar charts and scatterplots are helpful for seeing numerical and categorical patterns, other tools exist such that they make for meaningful visualization outside of the dataframe environment. Take, for example, a mapping tool! Since our primary objective is analyzing the relationship between levels in AQI and socioeconomic factors and/or race, it only makes sense to utilize such a tool. Enter GeoPandas, which is a powerful library tool for geospatial analysis, specifically turning geometrical data into one that can be rendered into a map. This tool is used widely in Geospatial Information Science (GIS), an emerging field which uses spatial data to make informed decisions in urban planning, public health, climate change mitigation efforts, and many more! Combine this with other data and you have yourself an informative map.

In [57]:
gdf = gpd.read_file("c_05mr24.shp")  

In [58]:
gdf = gdf.rename(columns={
    "COUNTYNAME": "county Name",  
    "FIPS": "fips",
})
gdf["fips"] = gdf["fips"].astype(int)

In [59]:
# Merge AQI data with county shapefile
gdf = gdf.merge(yearly_avg_aqi_df_wfips, on="fips", how="inner")
gdf = gdf.drop(columns=["STATE_y", "county Name_y"])
gdf = gdf.rename(columns={"STATE_x": "STATE", "county Name_x": "COUNTY NAME"})
gdf.set_crs(epsg=4326, inplace=True) 

,STATE,CWA,COUNTY NAME,fips,TIME_ZONE,FE_AREA,LON,LAT,geometry,State Name,Yearly Average AQI
0,ME,CAR,Washington,23029,E,se,-67.6361,45.0363,"MULTIPOLYGON (((-67.93539 44.40382, -67.93643 ...",Maine,33.968753
1,CT,BOX,Hartford,9003,E,nn,-72.7328,41.8064,"POLYGON ((-72.89799 42.03781, -72.8625 42.0377...",Connecticut,44.821179
2,CT,BOX,Tolland,9013,E,nn,-72.3365,41.8550,"POLYGON ((-72.50919 42.03411, -72.4833 42.0339...",Connecticut,41.312231
3,CT,BOX,Windham,9015,E,nn,-71.9875,41.8299,"POLYGON ((-72.0635 42.02731, -72.0425 42.02741...",Connecticut,36.432904
4,MD,LWX,Garrett,24023,E,ww,-79.2739,39.5286,"POLYGON ((-78.9906 39.72251, -78.9313 39.72271...",Maryland,39.958058
...,...,...,...,...,...,...,...,...,...,...,...
965,HI,HFO,Oahu in Honolulu,15003,H,None,-157.9765,21.4625,"MULTIPOLYGON (((-157.71642 21.28741, -157.7149...",Hawaii,28.195949
966,NY,BUF,Erie,36029,E,ww,-78.7308,42.7621,"MULTIPOLYGON (((-78.96736 42.71326, -78.96956 ...",New York,46.873291
967,NY,OKX,New York (Manhattan),36061,E,se,-73.9671,40.7786,"MULTIPOLYGON (((-74.04328 40.6897, -74.04449 4...",New York,45.812257
968,MD,LWX,Cecil,24015,E,ne,-76.0103,39.5606,"MULTIPOLYGON (((-75.81926 39.38305, -75.81705 ...",Maryland,45.938426


## Average AQI per County in California

Run the cell below and hover over each county to see their yearly average AQI.

In [60]:
gdf_ca = gdf[gdf["STATE"] == "CA"].copy()

gdf_ca = gdf_ca.set_crs(epsg=4326) if gdf_ca.crs is None else gdf_ca.to_crs(epsg=4326)
gdf_ca["geometry"] = gdf_ca["geometry"].simplify(0.001, preserve_topology=True)
gdf_ca["Yearly Average AQI"] = pd.to_numeric(gdf_ca["Yearly Average AQI"], errors="coerce")
gdf_ca = gdf_ca.dropna(subset=["Yearly Average AQI"])  # Remove NaNs

color_scale = branca.colormap.linear.YlOrRd_09.scale(
    gdf_ca["Yearly Average AQI"].min(), gdf_ca["Yearly Average AQI"].max()
)

m = folium.Map(location=[37.5, -119.5], zoom_start=6)

def style_function(feature):
    aqi_value = feature["properties"]["Yearly Average AQI"]
    return {
        "fillColor": color_scale(aqi_value) if aqi_value is not None else "gray",
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.7
    }

folium.GeoJson(
    gdf_ca,
    name="Counties",
    tooltip=folium.GeoJsonTooltip(
        fields=["COUNTY NAME", "Yearly Average AQI"],
        aliases=["County:", "AVG AQI (Yearly):"],  
        localize=True
    ),
    style_function=style_function
).add_to(m)

color_scale.caption = "Yearly Average AQI"
color_scale.add_to(m)
m

## Average AQI per County in Arizona

Our next study area is the State of Arizona. Do the same thing as above.

In [61]:
gdf_az = gdf[gdf["STATE"] == "AZ"].copy()

gdf_az = gdf_az.set_crs(epsg=4326) if gdf_az.crs is None else gdf_az.to_crs(epsg=4326)

gdf_az["geometry"] = gdf_az["geometry"].simplify(0.001, preserve_topology=True)

gdf_az["Yearly Average AQI"] = pd.to_numeric(gdf_az["Yearly Average AQI"], errors="coerce")
gdf_az = gdf_az.dropna(subset=["Yearly Average AQI"])  

color_scale = branca.colormap.linear.YlOrRd_09.scale(
    gdf_az["Yearly Average AQI"].min(), gdf_az["Yearly Average AQI"].max()
)

m = folium.Map(location=[34.0, -111.5], zoom_start=7.2)

def style_function(feature):
    aqi_value = feature["properties"]["Yearly Average AQI"]
    return {
        "fillColor": color_scale(aqi_value) if aqi_value is not None else "gray",
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.8
    }

folium.GeoJson(
    gdf_az,
    name="Counties",
    tooltip=folium.GeoJsonTooltip(
        fields=["COUNTY NAME", "Yearly Average AQI"],
        aliases=["County:", "AVG AQI (Yearly):"],  
        localize=True
    ),
    style_function=style_function
).add_to(m)

color_scale.caption = "Yearly Average AQI"
color_scale.add_to(m)

m

## NEXT STEPS (For Modules Developer):
- Aquire health data for counties in California and Arizona.
- Aquire socioeconomic data for both states.
- Analyze intersection
- Get input from professors